<a href="https://colab.research.google.com/github/zyz314/100-Days-Of-ML-Code/blob/master/project6_4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# testproject.py

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import util
from util import get_data, plot_data

import indicators as ind
from indicators import work_all_indicators, plot_indicators

import marketsimcode as mc
from marketsimcode import market_simulator, generate_statistics


import TheoreticallyOptimalStrategy as tos
from TheoreticallyOptimalStrategy import testPolicy, get_stock_data


def author():
  return "yzheng438"


def main():
    """
    主函数，执行整个项目流程
    """
    # 设置股票符号和日期范围
    symbol = 'JPM'
    start_date = '2008-01-01'
    end_date = '2009-12-31'
    start_value=100000
    initial_cash=100000

    # 获取股票数据
    stock_data = get_stock_data(symbol, start_date, end_date)


    # 运行所有技术指标并生成图表
    indicators = work_all_indicators(stock_data)
    ind.plot_indicators(stock_data, indicators)


    # 计算理论上的最佳交易策略
    trades = testPolicy(symbol, start_date, end_date, start_value)

    # 执行市场模拟
    portfolio_values = market_simulator(trades, initial_cash)

    # 生成统计数据和图表
    cumulative_return, mean_daily_return, std_daily_return, sharpe_ratio = generate_statistics(portfolio_values)

    # 打印统计数据
    print("Cumulative Return:", cumulative_return)
    print("Mean Daily Return:", mean_daily_return)
    print("Standard Deviation of Daily Return:", std_daily_return)
    print("Sharpe Ratio:", sharpe_ratio)

     # 获取基准数据
    benchmark = get_data([symbol], pd.date_range(start_date, end_date))
    benchmark = benchmark[symbol] * 1000  # 假设买入1000股并持有

    # 生成并保存投资组合价值的图表
    benchmark_values = portfolio_values['benchmark']
    portfolio_values = portfolio_values['portfolio']


    # 计算基准统计数据
    benchmark_cum_return = (benchmark[-1] / benchmark[0]) - 1
    benchmark_daily_returns = benchmark.pct_change().dropna()
    benchmark_mean_daily_return = benchmark_daily_returns.mean()
    benchmark_std_daily_return = benchmark_daily_returns.std()


    plt.figure(figsize=(10, 6))
    plt.plot(portfolio_values.index, portfolio_values / portfolio_values.iloc[0], label='Portfolio', color='blue')
    plt.plot(benchmark_values.index, benchmark_values / benchmark_values.iloc[0], label='Benchmark', color='red')
    plt.title('Portfolio vs. Benchmark')
    plt.xlabel('Date')
    plt.ylabel('Normalized Value')
    plt.legend(loc='best')
    plt.grid(True)
    plt.savefig('Figure_6_Portfolio_vs_Benchmark.png')
    plt.close()

if __name__ == "__main__":
    main()